In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.linear_model import Ridge, RidgeCV, ElasticNet, LassoCV, LassoLarsCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn import svm
import lightgbm as lgb
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
import matplotlib
import matplotlib.pyplot as plt
from scipy.stats import skew
from scipy.stats.stats import pearsonr
from scipy import stats
import xgboost as xgb
from scipy.stats import norm
#from pyglmnet import GLM # Marco: need to understand how to install this 
from sklearn.preprocessing import StandardScaler
from subprocess import call
from sklearn.cross_validation import KFold

from IPython import get_ipython
get_ipython().run_line_magic('matplotlib', 'inline')

/home/obaidur/software/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Create Ensemble class

In [2]:
class Ensemble(object):
    def __init__(self, n_folds, stacker, base_models,seed_value):
        self.n_folds = n_folds
        self.stacker = stacker
        self.base_models = base_models
        self.seed_value = seed_value
        
    def fit_predict(self, X, y, T):
        X = np.array(X)
        y = np.array(y)
        T = np.array(T)
        
        folds = list(KFold(len(y), n_folds=self.n_folds, shuffle=True, random_state=self.seed_value))
        #folds = KFold(n_splits=self.n_folds, shuffle=True, random_state=self.seed_value)
        
        print("folds=",folds)
        
        S_train = np.zeros((X.shape[0], len(self.base_models)))
        S_test = np.zeros((T.shape[0], len(self.base_models)))
        
        for i, reg in enumerate(self.base_models):
            S_test_i = np.zeros((T.shape[0], len(folds)))
            for j, (train_idx, test_idx) in enumerate(folds):
                X_train = X[train_idx]
                y_train = y[train_idx]
                X_holdout = X[test_idx]
                # y_holdout = y[test_idx]
                reg.fit(X_train, y_train)
                y_pred = reg.predict(X_holdout)[:]
                S_train[test_idx, i] = y_pred
                S_test_i[:, j] = reg.predict(T)[:]
            S_test[:, i] = S_test_i.mean(1)
        self.stacker.fit(S_train, y)
        y_pred = self.stacker.predict(S_test)[:]
        return y_pred


# Read in the dataset

In [3]:
    
seed = 2017
nfold = 5
bmodels = ["elasticnet","et","lgb","rf","xgb"]
    
train = pd.read_csv("../data/X_train_v1.csv")
y = train['SalePrice']
X = train.loc[:,'MSSubClass':'SaleCondition_Partial']
    
test = pd.read_csv("../data/X_test_v1.csv")
id = test["Id"]
T = test.loc[:,'MSSubClass':'SaleCondition_Partial']
    
#Set the base models

base_models_name = []
for j in range(len(bmodels)):
        modelname = ("../models/single/model_" + bmodels[j] + ".py")
        print(modelname)
        base_models_name.append(modelname)

print(base_models_name)
base_models = []
    
for i, bm in enumerate(base_models_name):
        model = !grep "model =" {bm}
        model = model[0]
        model = model[12:]
        model = eval(model)
        base_models.append(model)
#print(model)
#print(base_models) 

# Set the stacker model

xgb = xgb.XGBRegressor(n_estimators=250,learning_rate=0.1,max_depth=4,min_child_weight=1,objective='reg:linear')
# Call stacking
    
ens = Ensemble(n_folds=nfold, stacker=xgb,base_models=base_models,seed_value=seed)

../models/single/model_elasticnet.py
../models/single/model_et.py
../models/single/model_lgb.py
../models/single/model_rf.py
../models/single/model_xgb.py
['../models/single/model_elasticnet.py', '../models/single/model_et.py', '../models/single/model_lgb.py', '../models/single/model_rf.py', '../models/single/model_xgb.py']


In [4]:
    
results = ens.fit_predict(X,y,T)
print("results=",results)

folds= [(array([   0,    1,    2, ..., 1438, 1439, 1440]), array([  17,   20,   25,   41,   54,   55,   56,   57,   61,   64,   93,
        100,  102,  114,  115,  116,  117,  119,  120,  123,  128,  160,
        165,  166,  168,  179,  183,  188,  189,  191,  192,  196,  213,
        216,  222,  230,  232,  235,  238,  239,  243,  257,  261,  262,
        263,  270,  271,  273,  277,  282,  296,  299,  300,  313,  317,
        319,  323,  332,  333,  350,  353,  355,  356,  358,  371,  377,
        379,  383,  389,  391,  397,  400,  401,  411,  416,  421,  423,
        426,  430,  434,  436,  441,  443,  451,  452,  457,  461,  462,
        463,  464,  465,  467,  472,  476,  480,  497,  499,  512,  516,
        534,  538,  539,  544,  546,  548,  550,  554,  558,  561,  563,
        575,  585,  609,  618,  621,  623,  632,  635,  636,  638,  640,
        644,  647,  650,  652,  656,  657,  662,  668,  669,  674,  679,
        684,  690,  693,  701,  702,  710,  711,  712,  714,  721

In [ ]:
results = np.expm1(results)

In [18]:
print("results=",results)
results = pd.DataFrame(results)
results = pd.concat([id,results], axis=1)
results.columns =["Id","SalePrice"]
print("results=",results)
results.to_csv("./ensembled_results.csv", index = False)

results= [ 125759.921875  157115.3125    180642.625    ...,  168349.90625
  118038.078125  236046.65625 ]
results=         Id      SalePrice
0     1461  125759.921875
1     1462  157115.312500
2     1463  180642.625000
3     1464  197714.406250
4     1465  192241.250000
5     1466  167046.000000
6     1467  189112.125000
7     1468  168593.812500
8     1469  194693.656250
9     1470  125454.218750
10    1471  186688.187500
11    1472   94655.039062
12    1473   96320.859375
13    1474  153493.734375
14    1475  106779.976562
15    1476  392270.031250
16    1477  245875.406250
17    1478  248936.296875
18    1479  282734.750000
19    1480  514457.093750
20    1481  341050.906250
21    1482  206747.187500
22    1483  180842.562500
23    1484  164919.515625
24    1485  176022.875000
25    1486  197346.687500
26    1487  322820.406250
27    1488  233867.781250
28    1489  196300.968750
29    1490  229678.796875
...    ...            ...
1429  2890   75054.468750
1430  2891  139185.453125
1

In [16]:
results

array([ 125759.921875,  157115.3125  ,  180642.625   , ...,  168349.90625 ,
        118038.078125,  236046.65625 ], dtype=float32)

In [17]:
id

0       1461
1       1462
2       1463
3       1464
4       1465
5       1466
6       1467
7       1468
8       1469
9       1470
10      1471
11      1472
12      1473
13      1474
14      1475
15      1476
16      1477
17      1478
18      1479
19      1480
20      1481
21      1482
22      1483
23      1484
24      1485
25      1486
26      1487
27      1488
28      1489
29      1490
        ... 
1429    2890
1430    2891
1431    2892
1432    2893
1433    2894
1434    2895
1435    2896
1436    2897
1437    2898
1438    2899
1439    2900
1440    2901
1441    2902
1442    2903
1443    2904
1444    2905
1445    2906
1446    2907
1447    2908
1448    2909
1449    2910
1450    2911
1451    2912
1452    2913
1453    2914
1454    2915
1455    2916
1456    2917
1457    2918
1458    2919
Name: Id, dtype: int64